In [ ]:
%matplotlib inline
import numpy as np

# Gaussian elimination

Here is an implementation of Gaussian elimination without pivoting.

In [ ]:
def LU(A):
    """Factor a square matrix A into triangular matrices L, U via Gaussian elimination."""
    m = A.shape[0]
    U = A.copy()
    L = np.eye(m)
    for j in range(m):
        for i in range(j+1,m):
            L[i,j] = U[i,j]/U[j,j]
            U[i,:] = U[i,:] - L[i,j]*U[j,:]
    return L, U

Let's test it on a $3 \times 3$ matrix:

In [ ]:
A = np.array([[1.,2,1],[2,5,3],[1,4,9]])
L, U = LU(A)
print L
print U

The factors are triangular, as desired.  Does their product give $A$?

In [ ]:
A-np.dot(L,U)

Yes, it does.

Next, let us investigate backward stability of this algorithm.  If Gaussian elimination is backward stable, then our computed factors $L, U$ should satisfy

$$LU = A + \delta A$$

for some matrix $\delta A$ such that $\|\delta A\|/\|A\| \approx \epsilon_{machine} \approx 10^{-16}$.  In other words, we should have

$$\frac{\|LU - A \|}{\|A\|} \approx 10^{-16}.$$

The following code computes this quantity for 100 random matrices of size $50 \times 50$, and stops if it finds one that substantially violates the condition above.

In [ ]:
m = 50
for i in range(100):
    A = np.random.randn(m,m)
    L, U = LU(A)
    rel_err = np.linalg.norm(A-np.dot(L,U))/np.linalg.norm(A)
    if rel_err > 1e-14:
        print rel_err
        print "Random matrix "+str(i+1)+" produced a backward error of " + str(rel_err)
        break

Play around with the tolerance and the matrix size above.

Clearly, Gaussian elimination (without pivoting) is not backward stable.

# Partial pivoting

What goes wrong in Gaussian elimination?  Sometimes, the diagonal entry in the next column is very small.  That means that the multipliers used to cancel the remaining entries in that column will be very large, leading to an amplification of roundoff errors.  This can be alleviated via *pivoting*, which means choosing the largest entry in the next column and moving it to the diagonal.

Here is an implementation of Gaussian Elimination with partial pivoting.

In [ ]:
def PLU(A):
    """Factor a square matrix via Gaussian elimination with partial pivoting."""
    m = A.shape[0]
    U = A.copy()
    L = np.eye(m)
    P = np.eye(m)
    for j in range(m):
        ii = np.argmax(np.abs(U[j:,j]))+j
        #temp = U[j,:].copy(); U[j,:] = U[ii,:]; U[ii,:] = temp # This would also work.
        U[[j,ii],:] = U[[ii,j],:]  # Swap two rows via advanced slicing!
        P[[j,ii],:] = P[[ii,j],:]
        L[[j,ii],:j] = L[[ii,j],:j]
        
        for i in range(j+1,m):
            L[i,j] = U[i,j]/U[j,j]
            U[i,:] = U[i,:] - L[i,j]*U[j,:]
    return P, L, U

Let's check that it works:

In [ ]:
A = np.array([[1.,2,1],[2,5,3],[1,4,9]])
P, L, U = PLU(A)
print L
print U
print P

The outputs should satisfy $PA = LU$:

In [ ]:
print np.dot(P,A)
print np.dot(L,U)

Now, lets see how large the backward error is with pivoting:

In [ ]:
m = 100
all_rel_errs = []
for i in range(100):
    A = np.random.randn(m,m)
    P, L, U = PLU(A)
    rel_err = np.linalg.norm(np.dot(P,A)-np.dot(L,U))/np.linalg.norm(A)
    all_rel_errs.append(rel_err)
    if rel_err > 1e-15:
        print rel_err
        print i
        break

In [ ]:
print np.max(np.array(rel_err))

# A final example

In [ ]:
m = 60
A = np.zeros((m,m))
for i in range(m):
    for j in range(m):
        if i==j:
            A[i,j] = 1.
        elif i>j:
            A[i,j] = -1.
        elif j==m-1:
            A[i,j] = 1.

In [ ]:
P, L, U = PLU(A)
np.linalg.norm(np.dot(P,A)-np.dot(L,U))/np.linalg.norm(A)